In [1]:
import os
import numpy as np
import pandas as pd
from recbole.quick_start import run_recbole
import time
import datetime
#tensor factorization
#source: https://tensorly.org/stable/user_guide/tensor_decomposition.html
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker

[sequential recommendation parameter guidance](https://recbole.io/docs/get_started/started/sequential.html)



In [2]:
path = "G:/My Drive/2021/Bias/sumo_simulation/"
os.chdir(path)

#get obs data matrix
data_path_lst = []
for i in os.listdir():
    if len(i) == 12 and '.csv' in i:
        data_path_lst.append(i)
#data_path_lst

## Tensor factorization test

In [11]:
test_dt = pd.read_csv(data_path_lst[0])
timestep_convert = lambda x: time.mktime(datetime.datetime.strptime(('2017_4_'+x[:-4]),
                                                                    "%Y_%m_%d_%H_%M").timetuple())
test_dt['date_time'] = test_dt['key'].apply(timestep_convert)

def convert_idx(selectcol, newcol, data):
    newdict = {}
    idx = 0
    for i in np.unique(data[selectcol]):
        newdict[i] = idx
        idx += 1
    add_val = lambda x: newdict[x]
    data[newcol] = data[selectcol].apply(add_val)

In [6]:
#convert data to 3d matrix
num_ids, num_taz, num_time = len(np.unique(test_dt.newid)), len(np.unique(test_dt.taz)), len(np.unique(test_dt.time))

In [20]:
convert_idx('newid', 'newidx', test_dt)
convert_idx('taz', 'tazidx', test_dt)
convert_idx('date_time', 'timeidx', test_dt)

In [33]:
test_mx = np.zeros((num_ids, num_taz, num_time))

for i, j, k in test_dt[['newidx', 'tazidx', 'timeidx']].values:
    test_mx[i][j][k] = 1

In [37]:
test_mx = tl.tensor(test_mx)

In [ ]:
factors = parafac(test_mx, rank=100)

In [ ]:
reconstructed_tensor = tl.kruskal_to_tensor(factors)

# Comparing the original tensor with the reconstructed tensor
reconstruction_error = tl.norm(test_mx - reconstructed_tensor) / tl.norm(tensor_data)
print(f"Reconstruction error: {reconstruction_error}")

## recbole-based
### Sequential model test

In [ ]:
#reference source: https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

In [3]:
#convet data to recbole data format
test_dt = pd.read_csv(data_path_lst[0])

#convert time info to timestamp
timestep_convert = lambda x: time.mktime(datetime.datetime.strptime(('2017_4_'+x[:-4]),
                                                                    "%Y_%m_%d_%H_%M").timetuple())
test_dt['date_time'] = test_dt['key'].apply(timestep_convert)

In [5]:
test_dt.head()

,newid,sum,key,taz,date,time,date_time
0,26,1,4_3_55_608,608,4,3:55,1.491303e+09
1,26,2,4_4_5_615,615,4,4:05,1.491304e+09
2,26,1,4_3_40_578,578,4,3:40,1.491302e+09
3,26,3,4_3_40_627,627,4,3:40,1.491302e+09
4,32,1,4_11_35_502,502,4,11:35,1.491331e+09


In [52]:
temp = test_dt[['newid','taz','date_time']].rename(columns={'newid': 'user_id:token', 
                                                     'taz': 'item_id:token', 
                                                     'date_time': 'timestamp:float'})
temp.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

## Model test

In [6]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.model.sequential_recommender import FPMC

In [39]:
#os.listdir('../SUMO_simulation/')

In [43]:
parameter_dict_FPMC = {
    'data_path': '../SUMO_simulation/',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    #'train_neg_sample_args': None,
    'neg_sampling':None,
    'eval_args':{
        'topk':30,
        'order':'TO'
    }
}


config =  Config(model='FPMC', dataset='recbox_data', config_dict=parameter_dict_FPMC)

#run_recbole(model='FPMC', dataset='recbox_data', config_dict=parameter_dict)
# Step 4: Initialization
init_seed(config['seed'], config['reproducibility'])
logger = init_logger(config)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

# Step 5: Training
model_FPMC = FPMC(config, train_data.dataset).to(config['device'])
trainer = Trainer(config, model_FPMC)

# Train and evaluate the model
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, saved=True)

24 Apr 16:52    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1089, in emit
    self.flush()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1069, in flush
    self.stream.flush()
OSError: [Errno 22] Invalid argument
Call stack:
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\traitlets\confi

  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\2220578975.py", line 23, in <module>
    train_data, valid_data, test_data = data_preparation(config, dataset)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\data\utils.py", line 208, in data_preparation
    logger.info(
Message: "[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]"
Arguments: ()
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
24 Apr 16:52    INFO  epoch 0 training [time: 6.75s, train loss: 50.7117]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1089, in emit
    self.flush()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1069, in flu

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\2220578975.py", line 30, in <module>
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, saved=True)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\trainer\trainer.py", line 493, in fit
    self.logger.info(valid_score_output)
Message: 'epoch 0 evaluating [time: 0.19s, valid_score: 0.477800]'
Arguments: ()
epoch 0 evaluating [time: 0.19s, valid_score: 0.477800]
24 Apr 16:52    INFO  valid result: 
recall@10 : 0.6105    mrr@10 : 0.4778    ndcg@10 : 0.5096    hit@10 : 0.6105    precision@10 : 0.061
--- Logging error ---
Traceback (most

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3269, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\2220578975.py", line 30, in <module>
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, saved=True)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\trainer\trainer.py", line 502, in fi

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3009, in run_cell
    result = self._run_cell(
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3064, in _run_cell
    result = runner(coro)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3269, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, sel

...

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\2220578975.py", line 30, in <module>
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, saved=True)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\trainer\trainer.py", line 450, in fit
    self.logger.info(train_loss_output)
Message: 'epoch 23 training [time: 6.58s, train loss: 3.6886]'
Arguments: ()
epoch 23 training [time: 6.58s, train loss: 3.6886]
24 Apr 16:55    INFO  epoch 23 evaluating [time: 0.24s, valid_score: 0.538400]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Yiran\anaconda3\envs\modeltest\

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3269, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\2220578975.py", line 30, in <module>
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, saved=True)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\trainer\trainer.py", line 494, in fit
    self.logger.info(valid_result_output)
Message: 'valid result: \nrecall@10 : 0.7798    mrr@10 : 0.5384    ndcg@10 : 0.5962    hit@10 : 0.7798    prec

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3064, in _run_cell
    result = runner(coro)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3269, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\2220578975.py", line 30, in <module>
    best_valid_score, best_val

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\zmqshell.py", line 549, in run_cell
    return super().run_cell(*args, **kwargs)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3009, in run_cell
    result = self._run_cell(
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3064, in _run_cell
    result = runner(coro)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3269, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, as

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py", line 449, in do_execute
    res = shell.run_cell(
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\zmqshell.py", line 549, in run_cell
    return super().run_cell(*args, **kwargs)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3009, in run_cell
    result = self._run_cell(
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3064, in _run_cell
    result = runner(coro)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3269, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "C:\Users\Yiran\AppData\R

In [60]:
model_save_path = config['checkpoint_dir'] + '/model_final.pth'

torch.save(model.state_dict(), model_save_path)

In [53]:
import torch

In [55]:
user = torch.tensor([test_dt.newid[0]])  # User ID in tensor
item = torch.tensor([test_dt.taz[0]])

# Item ID in tensor
#model_FPMC

In [58]:
model_FPMC.

168

In [57]:
model_FPMC.predict(user, item)

TypeError: predict() takes 2 positional arguments but 3 were given

In [65]:
parameter_dict_SASRec = {
    'data_path': '../SUMO_simulation/',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'train_neg_sample_args': None,
    'neg_sampling':None,
    'save_model': True,  # This will save the model after training
    'checkpoint_dir': './model_checkpoints',
    'eval_args':{
        'topk':30,
        'order':'TO'
    }
}


config2 = Config(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)

run_recbole(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)

24 Apr 17:10    INFO  ['C:\\Users\\Yiran\\AppData\\Roaming\\Python\\Python38\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Yiran\\AppData\\Roaming\\jupyter\\runtime\\kernel-dabb97f1-c017-4e6d-a60c-5589f77aa5fc.json']
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1089, in emit
    self.flush()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1069, in flush
    self.stream.flush()
OSError: [Errno 22] Invalid argument
Call stack:
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Yiran\AppData\Roamin

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\tornado\platform\asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\base_events.py", line 570, in run_forever
    self._run_once()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\base_events.py", line 1859, in _run_once
    handle._run()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelbase.py", line 545, in dispatch_queue
    await self.process_one()
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelbase.py", line 534, in process_one
    await dispatch(*args)
  File "C:\Users\Yiran\AppDa

24 Apr 17:10    INFO  recbox_data
The number of users: 13710
Average actions of users: 16.57276241884893
The number of items: 168
Average actions of items: 1360.4550898203593
The number of inters: 227196
The sparsity of the dataset: 90.13597999374805%
Remain Fields: ['user_id', 'item_id', 'timestamp']
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1089, in emit
    self.flush()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1069, in flush
    self.stream.flush()
OSError: [Errno 22] Invalid argument
Call stack:
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel_launcher.py", line 18, in <m

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\1060703303.py", line 20, in <module>
    run_recbole(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\quick_start\quick_start.py", line 133, in run_recbole
    train_data, valid_data, test_data = data_preparation(config, dataset)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\data\utils.py", line 199, in data_preparation
    logger.info(
Message: "[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynam

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\tornado\platform\asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\base_events.py", line 570, in run_forever
    self._run_once()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\base_events.py", line 1859, in _run_once
    handle._run()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelbase.py", line 545, in dispatch_queue
    await self.process_one()
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelbase.py", line 534, in process_one
    await dispatch(*args)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelbase.py", line 437, in dispatch_shell
    await result
  File "C:\Users\Yiran\App

...

  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\tornado\platform\asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\base_events.py", line 570, in run_forever
    self._run_once()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\base_events.py", line 1859, in _run_once
    handle._run()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\

  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\1060703303.py", line 20, in <module>
    run_recbole(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\quick_start\quick_start.py", line 148, in run_recbole
    best_valid_score, best_valid_result = trainer.fit(
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\trainer\trainer.py", line 493, in fit
    self.logger.info(valid_score_output)
Message: 'epoch 62 evaluating [time: 8.04s, valid_score: 0.544100]'
Arguments: ()
epoch 62 evaluating [time: 8.04s, valid_score: 0.544100]
24 Apr 22:19    INFO  valid result: 
recall@10 : 0.7867    mrr@10 : 0.5441    ndcg@10 : 0.6022    hit@10 : 0.7867    precision@10 : 0.0787
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init__.py", line 1089, in emit
    self.flush()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\logging\__init_

  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3269, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3448, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Yiran\AppData\Local\Temp\ipykernel_2128\1060703303.py", line 20, in <module>
    run_recbole(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)
  File "c:\users\yiran\desktop\yiranzhang\recbole\recbole\quick_start\quick_start.py", line 14

  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\base_events.py", line 570, in run_forever
    self._run_once()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\base_events.py", line 1859, in _run_once
    handle._run()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelbase.py", line 545, in dispatch_queue
    await self.process_one()
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelbase.py", line 534, in process_one
    await dispatch(*args)
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelbase.py", line 437, in dispatch_shell
    await result
  File "C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py", line 362, in execute_request
    await super().execute_request(stream, ident, parent)
 

{'best_valid_score': 0.5463,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.7882),
              ('mrr@10', 0.5463),
              ('ndcg@10', 0.6043),
              ('hit@10', 0.7882),
              ('precision@10', 0.0788)]),
 'test_result': OrderedDict([('recall@10', 0.6934),
              ('mrr@10', 0.4535),
              ('ndcg@10', 0.5106),
              ('hit@10', 0.6934),
              ('precision@10', 0.0693)])}

device(type='cpu')

In [79]:
init_seed(config['seed'], config['reproducibility'])

In [16]:
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)

23 Apr 11:45    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../SUMO_simulation/recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset 

In [34]:
dataset = create_dataset(config2)
logger.info(dataset)
train_data, valid_data, test_data = data_preparation(config, dataset)

23 Apr 21:32    INFO  recbox_data
The number of users: 13710
Average actions of users: 16.57276241884893
The number of items: 168
Average actions of items: 1360.4550898203593
The number of inters: 227196
The sparsity of the dataset: 90.13597999374805%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 13710
Average actions of users: 16.57276241884893
The number of items: 168
Average actions of items: 1360.4550898203593
The number of inters: 227196
The sparsity of the dataset: 90.13597999374805%
Remain Fields: ['user_id', 'item_id', 'timestamp']
23 Apr 21:32    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
23 Apr 21:32    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{

In [35]:
from recbole.model.sequential_recommender import SASRec, GRU4Rec

In [36]:
# model loading and initialization
model = SASRec(config2, train_data.dataset).to(config2['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

23 Apr 21:33    INFO  SASRec(
  (item_embedding): Embedding(168, 64, padding_idx=0)
  (position_embedding): Embedding(50, 64)
  (trm_encoder): TransformerEncoder(
    (layer): ModuleList(
      (0-1): 2 x TransformerLayer(
        (multi_head_attention): MultiHeadAttention(
          (query): Linear(in_features=64, out_features=64, bias=True)
          (key): Linear(in_features=64, out_features=64, bias=True)
          (value): Linear(in_features=64, out_features=64, bias=True)
          (softmax): Softmax(dim=-1)
          (attn_dropout): Dropout(p=0.5, inplace=False)
          (dense): Linear(in_features=64, out_features=64, bias=True)
          (LayerNorm): LayerNorm((64,), eps=1e-12, elementwise_affine=True)
          (out_dropout): Dropout(p=0.5, inplace=False)
        )
        (feed_forward): FeedForward(
          (dense_1): Linear(in_features=64, out_features=256, bias=True)
          (dense_2): Linear(in_features=256, out_features=64, bias=True)
          (LayerNorm): LayerNo

24 Apr 01:50    INFO  epoch 53 training [time: 280.64s, train loss: 182.0153]
epoch 53 training [time: 280.64s, train loss: 182.0153]
24 Apr 01:50    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 01:54    INFO  epoch 54 training [time: 282.00s, train loss: 181.8769]
epoch 54 training [time: 282.00s, train loss: 181.8769]
24 Apr 01:54    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 01:59    INFO  epoch 55 training [time: 281.05s, train loss: 181.8251]
epoch 55 training [time: 281.05s, train loss: 181.8251]
24 Apr 01:59    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 02:04    INFO  epoch 56 training [time: 280.89s, train loss: 181.6839]
epoch 56 training [time: 280.89s, train loss: 181.6839]
24 Apr 02:04    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving curre

24 Apr 04:16    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 04:21    INFO  epoch 85 training [time: 285.41s, train loss: 180.3197]
epoch 85 training [time: 285.41s, train loss: 180.3197]
24 Apr 04:21    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 04:26    INFO  epoch 86 training [time: 292.41s, train loss: 180.4908]
epoch 86 training [time: 292.41s, train loss: 180.4908]
24 Apr 04:26    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 04:31    INFO  epoch 87 training [time: 294.10s, train loss: 180.3565]
epoch 87 training [time: 294.10s, train loss: 180.3565]
24 Apr 04:31    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 04:35    INFO  epoch 88 training [time: 279.15s, train loss: 180.3680]
epoch 88 trainin

24 Apr 06:47    INFO  epoch 116 training [time: 287.62s, train loss: 179.6055]
epoch 116 training [time: 287.62s, train loss: 179.6055]
24 Apr 06:47    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 06:52    INFO  epoch 117 training [time: 282.38s, train loss: 179.5195]
epoch 117 training [time: 282.38s, train loss: 179.5195]
24 Apr 06:52    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 06:57    INFO  epoch 118 training [time: 280.44s, train loss: 179.5949]
epoch 118 training [time: 280.44s, train loss: 179.5949]
24 Apr 06:57    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 07:02    INFO  epoch 119 training [time: 280.50s, train loss: 179.5956]
epoch 119 training [time: 280.50s, train loss: 179.5956]
24 Apr 07:02    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Savi

24 Apr 11:40    INFO  epoch 178 training [time: 283.63s, train loss: 178.4063]
epoch 178 training [time: 283.63s, train loss: 178.4063]
24 Apr 11:40    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 11:44    INFO  epoch 179 training [time: 282.91s, train loss: 178.5323]
epoch 179 training [time: 282.91s, train loss: 178.5323]
24 Apr 11:44    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 11:49    INFO  epoch 180 training [time: 281.58s, train loss: 178.5496]
epoch 180 training [time: 281.58s, train loss: 178.5496]
24 Apr 11:49    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 11:54    INFO  epoch 181 training [time: 281.04s, train loss: 178.4223]
epoch 181 training [time: 281.04s, train loss: 178.4223]
24 Apr 11:54    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Savi

RuntimeError: Parent directory saved does not exist.

    self.run()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\summary\writer\event_file_writer.py", line 244, in run
    

In [ ]:
# model loading and initialization
model2 = GRU4Rec(config2, train_data.dataset).to(config2['device'])
logger.info(model2)

# trainer loading and initialization
trainer = Trainer(config, model2)

# model training
best_valid_score_gru4rec, best_valid_result_gru4rec = trainer.fit(train_data)

self._run()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\summary\writer\event_file_writer.py", line 275, in _run
    self._record_writer.write(data)
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\summary\writer\record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 773, in write
    self.fs.append(self.filename, file_content, self.binary_mode)
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 167, in append
    self._write(filename, file_content, "ab" if binary_mode else "a")
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 171, in _write
    with io.open(filename, mode, encoding=encoding) as f:
OSError: [Errno 22] Invali

In [90]:
best_valid_score

-inf

In [89]:
best_valid_result

In [86]:
from recbole.utils.case_study import full_sort_topk
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [87]:
topk_items = []
for internal_user_id in list(range(dataset.user_num))[1:]:
    _, topk_iid_list = full_sort_topk([internal_user_id], model, test_data, k=12, device=config['device'])
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)
print(len(topk_items))

IndexError: index -1 is out of bounds for dimension 0 with size 0

In [88]:
external_item_str = [' '.join(x) for x in topk_items]
result = pd.DataFrame(external_user_ids, columns=['customer_id'])
result['prediction'] = external_item_str
result.head()

ValueError: Length of values (3) does not match length of index (13709)